In [1]:
import pandas as pd
import numpy as np
import time

First I will source relevant data from Basketball-reference

This first file will be the all-nba results from teams across NBA history. The data will be cleaned and melted in order to better merge it with my other data files. 

In [2]:
#downloaded data from basketball reference, only one table so not scraping required
all_nba = pd.read_csv('bbref_data/all_nba_dat.csv')

# Drop position designation from the name column
all_nba['P1'] = all_nba['P1'].astype(str)
all_nba['P1'] = all_nba['P1'].str[:-2]
all_nba['P2'] = all_nba['P2'].astype(str)
all_nba['P2'] = all_nba['P2'].str[:-2]
all_nba['P3'] = all_nba['P3'].astype(str)
all_nba['P3'] = all_nba['P3'].str[:-2]
all_nba['P4'] = all_nba['P4'].astype(str)
all_nba['P4'] = all_nba['P4'].str[:-2]
all_nba['P5'] = all_nba['P5'].astype(str)
all_nba['P5'] = all_nba['P5'].str[:-2]

#melt the data frame so each row is a given player
all_nba_melt = pd.melt(all_nba, id_vars=['Season','Tm'], value_vars=['P1', 'P2', 'P3', 'P4', 'P5'])
all_nba_melt.rename(columns={'value':'Player'}, inplace=True)
all_nba_melt.rename(columns={'Tm':'All-NBA'}, inplace=True)

#Now I will convert P1 to C, P2 and P3 to F, and P4 and P5 to G
all_nba_melt = all_nba_melt.replace(['P1'], 'C')
all_nba_melt = all_nba_melt.replace(['P2', 'P3'], 'F')
all_nba_melt = all_nba_melt.replace(['P4', 'P5'], 'G')
all_nba_melt.rename(columns={'variable':'Position'}, inplace=True)

#Now I will make so each season corresponds to the correct year
all_nba_melt['Season'] = all_nba_melt['Season'].astype(str)
all_nba_melt['Season'] = all_nba_melt['Season'].str[:4]
all_nba_melt['Season'] = all_nba_melt['Season'].astype(int) + 1
all_nba_melt.rename(columns={'Season':'year'}, inplace=True)

all_nba_melt.rename(columns={'All-NBA':'all_nba_tm'},inplace=True)

all_nba_melt.head()

,year,all_nba_tm,Position,Player
0,2023,1st,C,Joel Embiid
1,2023,2nd,C,Nikola Jokić
2,2023,3rd,C,Domantas Sabonis
3,2022,1st,C,Nikola Jokić
4,2022,2nd,C,Joel Embiid


Now I will scrape player statistics from basketball-reference from the years 1980-2023 (1980 is the year the 3 point line was introduced)

First I will scrape the basic box-score statistics of each player

In [3]:
# URL of the webpage to scrape
url = "https://www.basketball-reference.com/leagues/NBA_2023_per_game.html"

# Read the table from the URL
tables = pd.read_html(url)

# Extract the first table from the list of tables
df = tables[0]
df['year'] = 2023
years = list(range(1980, 2023))

# Create an empty list to store the dataframes
all_data = []

for year in years:
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    # Read the table from the URL
    tables = pd.read_html(url)

    # Extract the first table from the list of tables
    df_temp = tables[0]
    df_temp['year'] = year
    df = pd.concat([df, df_temp])
    #sleeping script in order to avoid 429 errors
    time.sleep(10)

bbref_basic_stats = df

Now I will do this for advanced statistics

In [4]:
# URL of the webpage to scrape
url = "https://www.basketball-reference.com/leagues/NBA_2023_advanced.html"

# Read the table from the URL
tables = pd.read_html(url)

# Extract the first table from the list of tables
df_adv = tables[0]
df_adv['year'] = 2023
years = list(range(1980, 2023))


for year in years:
    url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(year)
    # Read the table from the URL
    tables = pd.read_html(url)

    # Extract the first table from the list of tables
    df_temp = tables[0]
    df_temp['year'] = year
    df_adv = pd.concat([df_adv, df_temp])
    #sleeping script in order to avoid 429 errors
    time.sleep(10)

bbref_adv_stats = df_adv

Now I will also add wins as datapoint. I will also scrape this from basketball-reference.

In [84]:
# URL of the webpage to scrape
url = "https://www.basketball-reference.com/leagues/NBA_2023.html"

# Read the table from the URL
tables = pd.read_html(url)

# Extract the first table from the list of tables
tab1 = tables[0]
tab1.rename(columns={'Eastern Conference': 'Tm'}, inplace=True)
tab2 = tables[1]
tab2.rename(columns={'Western Conference': 'Tm'}, inplace=True)

#Now I will combine tab1 and tab2 vertically
df = pd.concat([tab1, tab2], axis=0)

#Now I will change the team column to the team abbreviations
#2023 had a slightly different format so I will have to do this differently than the other years
#removing "* (#)"
df['Tm'] = df['Tm'].str.replace(r"\*\s\(.*\)","",regex=True)
#removing "*"
df['Tm'] = df['Tm'].str.replace(r"\s\(.*\)","",regex=True)


#df['Tm'] = df['Tm'].map(team_abbreviations).fillna(df['Tm'])
df['year'] = 2023

years = list(range(1980, 2023))

for year in years:
    url = "https://www.basketball-reference.com/leagues/NBA_{}.html".format(year)
    # Read the table from the URL
    tables = pd.read_html(url)

    # Read the table from the URL
    tables = pd.read_html(url)

    # Extract the first table from the list of tables
    tab1 = tables[0]
    tab1.rename(columns={'Eastern Conference': 'Tm'}, inplace=True)
    tab2 = tables[1]
    tab2.rename(columns={'Western Conference': 'Tm'}, inplace=True)

    #Now I will combine tab1 and tab2 vertically
    df_tmp = pd.concat([tab1, tab2], axis=0)

    #Now I will change the team column to the team abbreviations
    df_tmp['Tm'] = df_tmp['Tm'].str.replace(r"*","",regex=True)

    df_tmp['year'] = year
    df = pd.concat([df, df_tmp])
    time.sleep(10)
wins = df

In [148]:
#I will map the teams to the the team abbreviations
wins = df

team_abbreviations = {
    'Milwaukee Bucks': 'MIL',
    'Boston Celtics': 'BOS',
    'Philadelphia 76ers': 'PHI',
    'Cleveland Cavaliers': 'CLE',
    'New York Knicks': 'NYK',
    'Brooklyn Nets': 'BKN',
    'Miami Heat': 'MIA',
    'Atlanta Hawks': 'ATL',
    'Toronto Raptors': 'TOR',
    'Chicago Bulls': 'CHI',
    'Indiana Pacers': 'IND',
    'Washington Wizards': 'WAS',
    'Orlando Magic': 'ORL',
    'Charlotte Hornets': 'CHA',
    'Detroit Pistons': 'DET',
    'Denver Nuggets': 'DEN',
    'Memphis Grizzlies': 'MEM',
    'Sacramento Kings': 'SAC',
    'Phoenix Suns': 'PHX',
    'Los Angeles Clippers': 'LAC',
    'Golden State Warriors': 'GSW',
    'Los Angeles Lakers': 'LAL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'Oklahoma City Thunder': 'OKC',
    'Dallas Mavericks': 'DAL',
    'Utah Jazz': 'UTA',
    'Portland Trail Blazers': 'POR',
    'Houston Rockets': 'HOU',
    'San Antonio Spurs': 'SAS',
    #Now going through teams that have moved/rebranded
    'Seattle SuperSonics': 'OKC',
    'Vancouver Grizzlies': 'MEM',
    'New Jersey Nets': 'BKN',
    'San Diego Clippers': 'LAC',
    'Kansas City Kings': 'SAC',
    'New Orleans Hornets': 'NOP',
    'Charlotte Bobcats': 'CHA',
    'Washington Bullets': 'WAS',
    'New Orleans/Oklahoma City Hornets': 'NOP'
}


wins['Tm'] = wins['Tm'].map(team_abbreviations).fillna(wins['Tm'])


In [149]:
#Now I will drop any team that has Division in its name
wins = wins[~wins['Tm'].str.contains('Division')]


Now I can begin to merge these 4 data-frames

In [150]:
bbref_basic_stats = bbref_basic_stats[['Player','year', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS']]
#found the data scraping error where headers were inserted into the data
# bbref_basic_stats[bbref_basic_stats.duplicated]
bbref_basic_stats = bbref_basic_stats[bbref_basic_stats['Player'] != 'Player']

bbref_adv_stats = bbref_adv_stats[['Player','year', 'Tm', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%',  'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]
bbref_adv_stats = bbref_adv_stats[bbref_adv_stats['Player'] != 'Player']

In [152]:
all_stat = pd.merge(bbref_basic_stats, bbref_adv_stats, on = ['Player', 'year','Tm'])
all_stat['Player'] = all_stat['Player'].str.replace('*', '',regex=True)

all_nba_melt = all_nba_melt[all_nba_melt['year']>=1980]

final_df = pd.merge(all_stat,all_nba_melt, on = ['Player', 'year'], how = "outer")
final_df.loc[final_df['all_nba_tm'].isnull(),'all_nba_tm'] = 'None'
final_df.loc[final_df['Position'].isnull(),'Position'] = final_df.loc[final_df['Position'].isnull(),'Pos']
final_df.fillna(0, inplace=True)

Now I must recode some of the team names in order to merge the wins data

In [153]:
#CHH denotes the charlotte hornets before they moved to New Orleans, this will be changed to NOP
final_df.loc[final_df['Tm']=='CHH','Tm'] = 'NOP'

# We will also change the New Orleans Hornets to NOP
final_df.loc[final_df['Tm']=='NOH','Tm'] = 'NOP'

#The first year in New Orleans, the team was coded as NOK, we will change this to NOP
final_df.loc[final_df['Tm'] == 'NOK','Tm'] = 'NOP'

#Change Charlotte Hornets to CHA to match with Bobcats designation
final_df.loc[final_df['Tm']=='CHO','Tm'] = 'CHA'

#We will also change the New Jersey Nets to BKN
final_df.loc[final_df['Tm']=='NJN','Tm'] = 'BKN'

final_df.loc[final_df['Tm']=='BRK','Tm'] = 'BKN'

#We will also change the Seattle Supersonics to OKC
final_df.loc[final_df['Tm']=='SEA','Tm'] = 'OKC'

#We will also change the Vancouver Grizzlies to MEM
final_df.loc[final_df['Tm']=='VAN','Tm'] = 'MEM'

#We will also change the San Diego Clippers to LAC
final_df.loc[final_df['Tm']=='SDC','Tm'] = 'LAC'

#We will also change the Kansas City Kings to SAC
final_df.loc[final_df['Tm']=='KCK','Tm'] = 'SAC'

#We will also change Washington Bullets to WAS
final_df.loc[final_df['Tm']=='WSB','Tm'] = 'WAS'

#We will also change PHO to PHX
final_df.loc[final_df['Tm']=='PHO','Tm'] = 'PHX'

#Charlotte Hornets became the New Orleans Hornets in 2003
wins.loc[(wins['Tm'] == 'CHA') & (wins['year'] < 2003), 'Tm'] = 'NOP'

In [154]:
nba_df = pd.merge(final_df, wins, on = ['Tm', 'year'], how = 'outer')
nba_df

,Player,year,Pos,Age,Tm,G,GS,MP,FG,FGA,...,VORP,all_nba_tm,Position,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Precious Achiuwa,2023,C,23,TOR,55,12,20.7,3.6,7.3,...,-0.1,None,C,41,41,0.5,17.0,112.9,111.4,1.59
1,OG Anunoby,2023,SF,25,TOR,67,67,35.6,6.3,13.2,...,1.5,None,SF,41,41,0.5,17.0,112.9,111.4,1.59
2,Dalano Banton,2023,PG,23,TOR,31,2,9.0,1.8,4.2,...,0.1,None,PG,41,41,0.5,17.0,112.9,111.4,1.59
3,Scottie Barnes,2023,SF,21,TOR,77,76,34.8,6.0,13.2,...,1.6,None,SF,41,41,0.5,17.0,112.9,111.4,1.59
4,Will Barton,2023,SG,32,TOR,16,2,13.2,1.8,4.9,...,0.0,None,SG,41,41,0.5,17.0,112.9,111.4,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23143,Daniel Theis,2022,C,29,BOS,21,6,18.7,3.2,5.3,...,0.2,None,C,51,31,0.622,2.0,111.8,104.5,7.02
23144,Brodric Thomas,2022,SG,25,BOS,12,0,5.0,0.7,1.5,...,0.0,None,SG,51,31,0.622,2.0,111.8,104.5,7.02
23145,Derrick White,2022,SG,27,BOS,26,4,27.4,3.6,8.8,...,0.2,None,SG,51,31,0.622,2.0,111.8,104.5,7.02
23146,Grant Williams,2022,PF,23,BOS,77,21,24.4,2.7,5.6,...,0.8,None,PF,51,31,0.622,2.0,111.8,104.5,7.02


In [155]:
nba_df.to_csv('merged_nba_data.csv')